In [1]:
from pynq import Overlay
from pynq import GPIO
import time

In [2]:
overlay = Overlay("/home/xilinx/cartridge.bit")
c = None

In [3]:
class Cartridge:
    def __init__(self):
        self.gpio_wr = GPIO(GPIO.get_gpio_pin(0), "out")
        self.gpio_rd = GPIO(GPIO.get_gpio_pin(1), "out")
        self.gpio_cs = GPIO(GPIO.get_gpio_pin(2), "out")
        self.gpio_rst = GPIO(GPIO.get_gpio_pin(3), "out")

        self.gpio_a = [GPIO(GPIO.get_gpio_pin(4 + i), "out") for i in range(16)]
        self.gpio_d = [GPIO(GPIO.get_gpio_pin(20 + i), "in") for i in range(8)]
        self.read_mode = True
        
        self.gpio_rst.write(0)
        time.sleep(0.1)
        self.gpio_rst.write(1)
        
    def _write_address(self, address):
        for i in range(16):
            self.gpio_a[i].write(address & 1)
            address = address >> 1
        time.sleep(1.0 / 1_000_000.0)
            
    def read(self, address, rom):
        self.gpio_rd.write(0)
        self.gpio_wr.write(1)
        self.gpio_cs.write(int(rom))
        
        self._write_address(address)
        
        if not self.read_mode:
            self.read_mode = True
            for x in self.gpio_d:
                x.release()
            self.gpio_d = [GPIO(GPIO.get_gpio_pin(20 + i), "in") for i in range(8)]
        
        data = 0
        for i in range(8):
            data |= self.gpio_d[i].read() << i
            
        self.gpio_cs.write(1)
            
        return data
            
    def write(self, address, data, rom):
        self.gpio_rd.write(1)
        self.gpio_wr.write(1)
        self.gpio_cs.write(int(rom))
        
        self._write_address(address)
        
        if self.read_mode:
            self.read_mode = False
            for x in self.gpio_d:
                x.release()
            self.gpio_d = [GPIO(GPIO.get_gpio_pin(20 + i), "out") for i in range(8)]
            
        for i in range(8):
            self.gpio_d[i].write(data & 1)
            data = data >> 1
            
        self.gpio_wr.write(0)
#         time.sleep(0.5 / 1_000_000.0)
        time.sleep(0.01)
        self.gpio_wr.write(1)
        self.gpio_rd.write(0)
        self.gpio_cs.write(1)
        
    def release(self):
        gpios = [self.gpio_wr, self.gpio_rd, self.gpio_cs, self.gpio_rst] + self.gpio_a + self.gpio_d
        for gpio in gpios:
            gpio.release()
        
if c is not None:
    c.release()
c = Cartridge()

In [7]:
cartridge_type = c.read(0x0147, True)
print("cart type", hex(cartridge_type))
rom_size = c.read(0x0148, True)
print("rom size", hex(rom_size))
ram_size = c.read(0x0149, True)
print("ram size ", hex(ram_size))

cart type 0x10
rom size 0x6
ram size  0x3


In [113]:
# Cartridge type
hex(c.read(0x0147, True))

'0x3'

In [114]:
hex(c.read(0x4000, True))

'0x5f'

In [96]:
# Rom bank number
c.write(0x2000, 5, True)

In [99]:
# Enable ram
c.write(0x0000, 0x0A, True)

In [103]:
hex(c.read(0xA000, False))

'0xff'

In [102]:
# Disable ram
c.write(0x0000, 0x00, True)

In [8]:
ram_dump = []

In [15]:
start = time.time()
# enable ram
c.write(0x0000, 0x0A, True)
# bank
# done: 0 1 2 not done: 3 
c.write(0x4000, 0x3, True)
time.sleep(1)
for i in range(0xA000, 0xA000 + (8 * 1024)):
    b = c.read(i, False)
    ram_dump.append(b)
# disable ram
c.write(0x0000, 0x00, True)
end = time.time()
print("completed in ", end - start)

print(c.read(0xA000, False))


completed in  71.19623327255249
255


In [17]:
with open('/home/xilinx/ram-silver-1.sav', 'wb') as f:
    f.write(bytes(ram_dump))

In [16]:
print(len(ram_dump))

32768
